In [20]:
import torch
import pickle
import tarfile
import os
import lzma
import random
#from transformers import GPT2Tokenizer
from tqdm import tqdm
import time
import torch.nn as nn
from torch.nn import functional as F
import nbimporter #import from jupyter notebook file
from training import Block, FeedFoward, MultiHeadAttention, Head, GPTLanguageModel # Replace with your actual import

data = read_text_files('processed_texts')
#tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# Add any other necessary imports here

Reading Text Files: 100%|██████████████████████████████████████████████████████████| 1000/1000 [00:38<00:00, 26.08it/s]


In [21]:
print('Loading pre-trained model...')
with open('model-01.pkl', 'rb') as f:
    model = pickle.load(f)

print(time.time())
print ('Model loaded successfully!')


Loading pre-trained model...
1700313235.3036556
Model loaded successfully!


In [22]:
def read_text_files(directory):
    texts = []
    # Retrieve the list of files in the directory
    filenames = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    # Loop through the files with a progress bar
    for filename in tqdm(filenames, desc="Reading Text Files"):
        filepath = os.path.join(directory, filename)
        with open(filepath, 'r', encoding='utf-8') as file:
            texts.append(file.read())
    return texts


In [23]:
class SimpleTokenizer:
    def __init__(self):
        self.token_to_id = {"<PAD>": 0, "<UNK>": 1}  # Initialize with special tokens
        self.id_to_token = {0: "<PAD>", 1: "<UNK>"}
        self.char_to_id = {}
        self.id_to_char = {}
    def build_vocab(self, texts):
        unique_tokens = set()
        for text in tqdm(texts, desc="Building Vocabulary"):
            unique_tokens.update(text.split())
        
        for token in unique_tokens:
            if token not in self.token_to_id:
                token_id = len(self.token_to_id)
                self.token_to_id[token] = token_id
                self.id_to_token[token_id] = token

    def encode(self, text):
        return [self.token_to_id.get(word, self.token_to_id["<UNK>"]) for word in text.split()]

    def decode(self, token_ids):
        return ' '.join(self.id_to_token.get(id, "<UNK>") for id in token_ids)


In [24]:
build_vocab = False #sett til true om du vil bygge vocab list
if(build_vocab):
    tokenizer = SimpleTokenizer()
    tokenizer.build_vocab(read_text_files('processed_texts'))
    with open('tokenizer_vocab.pkl', 'wb') as f:
        pickle.dump((tokenizer.char_to_id, tokenizer.id_to_char), f)


In [27]:
if build_vocab == False:
    with open('tokenizer_vocab.pkl', 'rb') as f:
        char_to_id, id_to_char = pickle.load(f)
        tokenizer.char_to_id = char_to_id
        tokenizer.id_to_char = id_to_char

tokenized_data = [tokenizer.encode(text) for text in tqdm(data, desc="Tokenizing")]



Tokenizing: 100%|██████████████████████████████████████████████████████████████████| 1000/1000 [06:48<00:00,  2.45it/s]
